In [33]:
Q_Loss = 5-4.86491
F= 96485.33212
z_sei = 2
V_neg = 8.52e-5 * 1.58 * 0.065
Sr = 2 * 2e-8*1.5e-8*3.18e15 + 1
V_SEI_bar = 9.85e-5
a_neg = 3*0.75/5.86e-06


In [54]:
aa = {}
aa["Neg"] = 1
print(aa)

{'Neg': 1}


In [36]:
delta_SEI = (
    3600*Q_Loss*V_SEI_bar
    /
    (
        F*z_sei *   Sr*a_neg*V_neg
    ) / 2
)
print(delta_SEI)
# print(delta_SEI+2.5e-9)
print(calculate_initial_SEI_thickness(4e-5))

1.2704316446291918e-08
5.15911499515315e-09


In [55]:
import pybamm as pb

In [70]:
# Write a version to insert into Para_init:

Para_0=pb.ParameterValues("OKane2023" )
print(Para_0["Initial outer SEI thickness [m]"],Para_0["Negative electrode porosity"] )
def Overwrite_Initial_L_SEI_0_Neg_Porosity(Para_0,cap_loss):
    """ 
    This is to overwrite the initial negative electrode porosity 
    and initial SEI thickness (inner, outer) to be consistent 
    with the initial capacity loss 
    """
    # cap_loss = 5 - 4.86491
    delta_Q_SEI = cap_loss * 3600
    V_SEI = Para_0["Outer SEI partial molar volume [m3.mol-1]"] 
    # do this when finish updating
    F = 96485.3
    A = Para_0["Electrode width [m]"] * Para_0["Electrode height [m]"]
    z_SEI = Para_0["Ratio of lithium moles to SEI moles"] 
    L_neg = Para_0["Negative electrode thickness [m]"] 
    eps_act_neg = Para_0["Negative electrode active material volume fraction"]
    R_neg =   Para_0["Negative particle radius [m]"]
    l_cr_init = Para_0["Negative electrode initial crack length [m]"]
    w_cr = Para_0["Negative electrode initial crack width [m]"]
    rho_cr = Para_0["Negative electrode number of cracks per unit area [m-2]"]
    a_neg = (3 * eps_act_neg / R_neg)
    roughness = 1 + 2 * l_cr_init * w_cr * rho_cr
    L_SEI_init = delta_Q_SEI  * V_SEI / (
        z_SEI * F * A * L_neg * a_neg * roughness)

    delta_epi = (L_SEI_init ) * roughness * a_neg
    L_inner_init = L_SEI_init / 2
    epi = 0.25 - delta_epi
    # print(L_inner_init,epi)
    # important: update here!
    Para_0["Negative electrode porosity"] = epi
    Para_0["Initial outer SEI thickness [m]"] = L_inner_init
    Para_0["Initial inner SEI thickness [m]"] = L_inner_init
    print(f"Has Overwritten Initial outer SEI thickness [m] to be {L_inner_init:.2e} and Negative electrode porosity to be {epi:.3f} to account for initial capacity loss of {cap_loss:.3f} Ah")

    return Para_0

cap_loss = 5 - 4.86491
Para_0 = Overwrite_Initial_L_SEI_0_Neg_Porosity(Para_0,cap_loss)

1.23625e-08 0.21289519
Has Overwritten Initial outer SEI thickness [m] to be 1.24e-08 and Negative electrode porosity to be 0.222 to account for initial capacity loss of 0.135 Ah


In [59]:
# from Simon:
def calculate_initial_SEI_thickness(V_SEI):
    delta_Q_SEI = 3.6 * (5000 - 4864.91)
    F = 96485.3
    A = 1.58 * 0.065
    z_SEI = 2
    L_neg = 8.52e-5
    eps_act_neg = 0.75
    R_neg = 5.86e-6
    l_cr_init = 2e-8
    w_cr = 1.5e-8
    rho_cr = 3.18e15
    a_neg = (3 * eps_act_neg / R_neg)
    roughness = 1 + 2 * l_cr_init * w_cr * rho_cr
    L_SEI_init = delta_Q_SEI  * V_SEI / (
        z_SEI * F * A * L_neg * a_neg * roughness
    )
    delta_epi = (L_SEI_init ) * roughness * a_neg
    L_inner_init = L_SEI_init / 2
    epi = 0.25 - delta_epi
    return L_inner_init,epi
L_inner_init,epi = calculate_initial_SEI_thickness(9.585e-5)
print(L_inner_init,epi )

1.2362529307135732e-08 0.22239316407624946
